# License Plate Detection - Data Collection and Exploration
Welcome to the traffic light detection! This notebook explains step-by-step how to train and use license plate detection models for the Machine Learning Engineer Capstone Project. Make sure you meet all preconditions before you start, see README.md.

### Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import matplotlib

from distutils.version import StrictVersion
from matplotlib import pyplot as plt
from PIL import Image

Test Tensorflow version...

In [3]:
print ("Tensorflow: {}".format(tf.__version__))

if StrictVersion(tf.__version__) < StrictVersion('1.13.1'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.13.1')

if StrictVersion(tf.__version__) >= StrictVersion('2.0.0'):
    raise ImportError('Please downgrade your TensorFlow installation to v1.13.*.')

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Tensorflow: 1.13.1
Default GPU Device: /device:GPU:0


### Env setup

In [4]:
# This is needed to display the images inside the notebook cells.
%matplotlib inline

os.chdir(os.path.join(os.getcwd(), 'tf_object_detection'))
working_dir = os.getcwd()

sys.path.append(working_dir)
sys.path.append(working_dir + "/slim")

path = working_dir + ';' + working_dir + '/slim' + ';' + working_dir + '/object_detection'
os.environ['PYTHONPATH'] = path

os.chdir(os.path.join(os.getcwd(), 'object_detection'))

### Object detection imports
Here are the imports from the object detection module.

In [5]:
import object_detection

### Model preparation

#### Variables
Define the pretrained model to be used.
Change MODEL and PIPELINE_CONFING_FILENAME if you want to use other pretrained models, see the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [6]:
# choose pretrained model
MODEL = 'ssdlite_mobilenet_v2_coco_2018_05_09'
PIPELINE_CONFING_FILENAME = 'ssdlite_mobilenet_v2_coco.config'

#MODEL = 'ssd_mobilenet_v2_coco_2018_03_29'
#PIPELINE_CONFING_FILENAME = 'ssd_mobilenet_v2.config'

NUM_CLASSES = 1

In [7]:
# ****************** don't edit ******************

# What model to download.
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PRETRAIND_MODELS_DIR = 'models'

# List of the strings that is used to add correct label for each box.
DETECTION_COFIG_DIR = '../../config/plate_detection'
PATH_TO_LABELS = os.path.join(DETECTION_COFIG_DIR, 'labels_map.pbtxt')

PIPELINE_CONFING_FILEPATH = os.path.join(DETECTION_COFIG_DIR, PIPELINE_CONFING_FILENAME)

OUTPUT_DIR = '../../output/plate_detection'
CHECKPOINTS_DIR = OUTPUT_DIR + '/checkpoints/' + MODEL
FINETUNED_MODEL_DIR = OUTPUT_DIR + '/fine_tuned_models/' + MODEL

# Path to frozen detection graph.
PATH_TO_FROZEN_GRAPH = os.path.join(FINETUNED_MODEL_DIR, 'frozen_inference_graph.pb')

TFLITE_MODEL_PATH = os.path.join(OUTPUT_DIR, 'glpd-model-float.tflite')

print ('Labels file:           {}'.format(PATH_TO_LABELS))
print ('Pipeline config file:  {}'.format(PIPELINE_CONFING_FILEPATH))
print ('Checkpoints directory: {}'.format(CHECKPOINTS_DIR))
print ('Frozen model file:     {}'.format(PATH_TO_FROZEN_GRAPH))
print ('TFLite model file:     {}'.format(TFLITE_MODEL_PATH))

Labels file:           ../../config/plate_detection\labels_map.pbtxt
Pipeline config file:  ../../config/plate_detection\ssdlite_mobilenet_v2_coco.config
Checkpoints directory: ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09
Frozen model file:     ../../output/plate_detection/fine_tuned_models/ssdlite_mobilenet_v2_coco_2018_05_09\frozen_inference_graph.pb
TFLite model file:     ../../output/plate_detection\glpd-model-float.tflite


# Train Traffic Light Detection Model

**NOTE:** If you have already trained the model, you can skip the training- and export steps and proceed directly with testing...

### Download Pretrained Model

**NOTE:** This step only needs to be done once if the pretrained model has not been downloaded yet!

In [10]:
MODEL_ZIP = os.path.join(PRETRAIND_MODELS_DIR, MODEL_FILE)
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_ZIP)
tar_file = tarfile.open(MODEL_ZIP)
tar_file.extractall(os.path.join(os.getcwd(), PRETRAIND_MODELS_DIR))
tar_file.close()

### Train Finetuned Model

Starts the training. This step may take several hours, depending on the computing power of your computer.
You can monitor the training process using tensorboard tensorboard --logdir=%CHECKPOINTS_DIR% and stop the training if once the desired accuracy has been achieved.

In [ ]:
%run model_main.py --pipeline_config_path={PIPELINE_CONFING_FILEPATH} --model_dir={CHECKPOINTS_DIR}


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_tpu False
INFO:tensorflow:Using config: {'_model_dir': '../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '

INFO:tensorflow:Restoring parameters from ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt-929
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.220
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.581
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.279
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.249
 Average Recal

### Export Finetuned Model
If the training is finished, the model must be exported so that it can be used.

Set CHECKPOINT_NO = 'XXX' where XXX is the number of the last checkpoint file in CHECKPOINTS_DIR, before you start the export.

In [8]:
CHECKPOINT_NO = 30000

!python export_inference_graph.py --pipeline_config_path={PIPELINE_CONFING_FILEPATH} --trained_checkpoint_prefix={CHECKPOINTS_DIR}/model.ckpt-{CHECKPOINT_NO} --output_directory={FINETUNED_MODEL_DIR}

Parsing Inputs...



W0318 06:56:41.540622 19104 deprecation_wrapper.py:119] From export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0318 06:56:41.544592 19104 deprecation_wrapper.py:119] From D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection\object_detection\exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0318 06:56:41.544592 19104 deprecation_wrapper.py:119] From D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection\object_detection\exporter.py:121: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0318 06:56:41.566620 19104 deprecation_wrapper.py:119] From D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection\object_detection\core\preprocessor.py:2937: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.


W0318 06:56:41.584622 19104 deprecation_wrapper.py:119] F



=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-account_type_regexes       _trainable_variables
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     params
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
param: Number of parameters (in the Variable).

Profile:
node name | # paramet

W0318 06:56:41.809620 19104 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x00000270BB0F2080>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x00000270BB0F2080>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0318 06:56:41.836621 19104 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x00000270BB269C50>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full o

W0318 06:56:43.058623 19104 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x00000270BB9CC518>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x00000270BB9CC518>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0318 06:56:43.104621 19104 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x00000270BB9CC470>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <

      BoxPredictor_1/BoxEncodingPredictor/biases (8, 8/8 params)
      BoxPredictor_1/BoxEncodingPredictor/weights (3x3x1280x8, 92.16k/92.16k params)
    BoxPredictor_1/ClassPredictor (--/46.08k params)
      BoxPredictor_1/ClassPredictor/biases (4, 4/4 params)
      BoxPredictor_1/ClassPredictor/weights (3x3x1280x4, 46.08k/46.08k params)
  BoxPredictor_2 (--/55.31k params)
    BoxPredictor_2/BoxEncodingPredictor (--/36.87k params)
      BoxPredictor_2/BoxEncodingPredictor/biases (8, 8/8 params)
      BoxPredictor_2/BoxEncodingPredictor/weights (3x3x512x8, 36.86k/36.86k params)
    BoxPredictor_2/ClassPredictor (--/18.44k params)
      BoxPredictor_2/ClassPredictor/biases (4, 4/4 params)
      BoxPredictor_2/ClassPredictor/weights (3x3x512x4, 18.43k/18.43k params)
  BoxPredictor_3 (--/27.66k params)
    BoxPredictor_3/BoxEncodingPredictor (--/18.44k params)
      BoxPredictor_3/BoxEncodingPredictor/biases (8, 8/8 params)
      BoxPredictor_3/BoxEncodingPredictor/weights (3x3x256x8, 18.

# Test Traffic Light Detection

Uses the fine tuned traffic light detection model FINETUNED_MODEL_DIR to detect traffic lights in images.

#### Load the (frozen) finetuned model into memory.

In [9]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

### Loading label map
Loads the label map file PATH_TO_LABELS. This file contains a map that assigns category names to the class-indices:

item { id: 1 name: 'Green' }

item { id: 2 name: 'Yellow' }

item { id: 3 name: 'Red' }

In [10]:
from object_detection.utils import label_map_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [11]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [12]:
import glob

PATH_TO_TEST_IMAGES_DIR = '../../data/plate_detection/test_images'

TEST_IMAGE_PATHS = []
for filename in glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg')):
    TEST_IMAGE_PATHS.append(filename)
    
print ('{} test images found in {}'.format(len(TEST_IMAGE_PATHS), PATH_TO_TEST_IMAGES_DIR))

5 test images found in ../../data/plate_detection/test_images


In [13]:
from object_detection.utils import visualization_utils as vis_util

# number of samples to test; -1 <=> all
NUM_SAMPLES = 2

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 12)

if NUM_SAMPLES > 0:
    test_images = np.random.choice(TEST_IMAGE_PATHS, size = NUM_SAMPLES, replace=False)
else:
    test_images = TEST_IMAGE_PATHS

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in test_images:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections], feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=4)
            plt.figure(figsize=IMAGE_SIZE)
            plt.title(image_path)
            plt.imshow(image_np)
            plt.show()

c:\users\andreas\anaconda3\envs\tf1.14\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


### Convert to TF Lite

In [15]:
from tensorflow.compat.v1.lite import TFLiteConverter, Optimize

input_arrays=["image_tensor"]
output_arrays=["detection_boxes","detection_scores","num_detections","detection_classes"]
input_tensor={"image_tensor":[1,300,300,3]}

converter = TFLiteConverter.from_frozen_graph(PATH_TO_FROZEN_GRAPH, input_arrays, output_arrays, input_tensor)
#converter = TFLiteConverter.from_saved_model(FINETUNED_MODEL_DIR + '/saved_model')
converter.optimizations=[Optimize.DEFAULT]
tflite_model = converter.convert()

open(TFLITE_MODEL_PATH, "wb").write(tflite_model)

ConverterError: TOCO failed. See console for info.
2020-03-18 06:58:35.038994: I tensorflow/lite/toco/import_tensorflow.cc:1336] Converting unsupported operation: Enter
2020-03-18 06:58:35.039329: E tensorflow/core/framework/op_kernel.cc:1426] OpKernel ('op: "NoOp" device_type: "CPU"') for unknown op: NoOp
2020-03-18 06:58:35.039530: E tensorflow/core/framework/op_kernel.cc:1426] OpKernel ('op: "NoOp" device_type: "GPU"') for unknown op: NoOp
2020-03-18 06:58:35.039689: E tensorflow/core/framework/op_kernel.cc:1426] OpKernel ('op: "_HostRecv" device_type: "GPU" host_memory_arg: "tensor"') for unknown op: _HostRecv
2020-03-18 06:58:35.039886: E tensorflow/core/framework/op_kernel.cc:1426] OpKernel ('op: "_Send" device_type: "CPU"') for unknown op: _Send
2020-03-18 06:58:35.040043: E tensorflow/core/framework/op_kernel.cc:1426] OpKernel ('op: "_HostRecv" device_type: "CPU"') for unknown op: _HostRecv
2020-03-18 06:58:35.040211: E tensorflow/core/framework/op_kernel.cc:1426] OpKernel ('op

In [30]:
from tensorflow.compat.v1.lite import Interpreter

# Load TFLite model and allocate tensors.
interpreter = Interpreter(model_path=PATH_TO_FROZEN_GRAPH)
interpreter.allocate_tensors()
# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on random input data
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']
print("Input Shape:  {}".format(input_shape))
print("Input Details: {}".format(input_details))
print("Output Details: {}".format(output_details))
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Output: {}".format(output_data))

ValueError: Model provided has model identifier 'imag', should be 'TFL3'
